In [1]:
import datetime, time
import simpy
import shapely.geometry
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import openclsim
import openclsim.core as core
import openclsim.plot as plot
import openclsim.model as model

import vo_colos.vodr_colos.vodr_core as vodr_core
import vo_colos.test_colos.test_core as test_core

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry

# 1. Define the environment

In [2]:
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time=time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

# 2. Define the sites

In [3]:
class Site(
        openclsim.core.Identifiable,
        openclsim.core.Log,
        vodr_core.Locatable,
        openclsim.core.HasContainer,
        openclsim.core.HasResource,
    ):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        pass

from_site_1 = Site(
    env=my_env,
    name="Winlocatie 1",
    lat = 4.1,
    lon = 52,
    level=50_000,
    capacity=50_000,
)

from_site_2 = Site(
    env=my_env,
    name="Winlocatie 2",
    lat = 4.2,
    lon = 52,
    level=150_000,
    capacity=150_000,
)

to_site = Site(
    env=my_env,
    name="Dumplocatie",
    lat = 4,
    lon= 52.1,
    capacity=200_000,
    level=0,
)

init
level: 50000
completed init
init
level: 150000
completed init
init
level: 0
completed init


# 3. Define the vessels

In [4]:
class LoadingFunction:
    def __init__(self, loading_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loading_rate = loading_rate

    def loading(self, origin, destination, amount):
        load_time = amount / self.loading_rate
        return load_time


class UnloadingFunction:
    def __init__(self,  *args, **kwargs):
        super().__init__(*args, **kwargs)

    def unloading(self, origin, destination, amount):
        dump_time = 10 * 60
        return dump_time

class TransportProcessingResource(
        openclsim.core.Identifiable,
        openclsim.core.Log,
        test_core.ContainerDependentMovable,
        openclsim.core.Processor,
        openclsim.core.HasResource,
        LoadingFunction,
        UnloadingFunction,
    ):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        pass

hopper = TransportProcessingResource(
    env=my_env,
    name="Hopper 01",
    lat=4,
    lon=52.1,
    loading_rate=1,
    capacity=5_000, 
    v_empty=4.5,
    v_full= 5,
)

init
level: 0
completed init


# 4. Define the activity

In [5]:
activity_1 = openclsim.model.Activity(
    env=my_env,
    name="Soil movement 1",
    origin=from_site_1,
    destination=to_site,
    loader=hopper,
    mover=hopper,
    unloader=hopper,
    start_event=None,
    stop_event=None,
)

activity_2 = openclsim.model.Activity(
    env=my_env,
    name="Soil movement 2",
    origin=from_site_2,
    destination=to_site,
    loader=hopper,
    mover=hopper,
    unloader=hopper,
    start_event=activity_1.main_process,
    stop_event=None,
)

start put_available
register new event 
start get_available


TypeError: conditional_process() got an unexpected keyword argument 'stop_event'

# 5. Run the Simulation

In [ ]:
my_env.run()

# 6. Simulations visualisation

In [ ]:
objects = [activity_1, activity_2, hopper, from_site_1, from_site_2, to_site]
activities = []
for obj in objects:
    L = set(obj.log["Message"])
    for l in L:
        if l[-5:] == 'start' and l[:-6] not in activities:
            activities.append(l[:-6])
            
C = np.linspace(0,255, len(activities))
colors = {}
for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

plot.vessel_planning(objects, activities, colors)

plt.figure(figsize=(14,5))